In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale

In [2]:
path = './UJIIndoorLoc/trainingData2.csv'
train_df = pd.read_csv(path,header=0)
train_ap_features = scale(np.asarray(train_df.iloc[:,:520]).astype(float),axis = 1)
train_df['REFPOINT'] = train_df.apply(lambda row: str(int(row['SPACEID'])) + str(int(row['RELATIVEPOSITION'])),
                                  axis=1)

In [3]:
blds = np.unique(train_df[['BUILDINGID']]) ##映射Building和Floor到REFPOINT的方法,减少编码量
flrs = np.unique(train_df[['FLOOR']])

for bld in blds:
    for flr in flrs:
        cond = (train_df['BUILDINGID'] == bld) & (train_df['FLOOR'] == flr)
        _, idx = np.unique(train_df.loc[cond, 'REFPOINT'], return_inverse=True)  # refer to numpy.unique manual
        train_df.loc[cond, 'REFPOINT'] = idx

blds = np.asarray(pd.get_dummies(train_df['BUILDINGID']))#3
flrs = np.asarray(pd.get_dummies(train_df['FLOOR']))#5
rfps = np.asarray(pd.get_dummies(train_df['REFPOINT']))#110
x_test_utm = (np.asarray(train_df['LONGITUDE'])).reshape(len(train_df),1)#1
y_test_utm = (np.asarray(train_df['LATITUDE'])).reshape(len(train_df),1)#1
train_labels = np.concatenate((blds, flrs, rfps,x_test_utm,y_test_utm), axis=1) #120
OUTPUT_DIM = train_labels.shape[1]-2  #118
# train_labels = np.concatenate((blds, flrs, rfps), axis=1)
# OUTPUT_DIM = train_labels.shape[1]

C:\Users\xfp\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [4]:
train_test_split = np.random.rand(len(train_ap_features)) < 0.9
x_train_val = train_ap_features[train_test_split]
y_train_val = train_labels[train_test_split]
x_test = train_ap_features[~train_test_split]
y_test = train_labels[~train_test_split]

train_val_split = np.random.rand(len(x_train_val)) < 0.9
x_train = x_train_val[train_val_split]
y_train = y_train_val[train_val_split]
x_val = x_train_val[~train_val_split]
y_val = y_train_val[~train_val_split]

# x_train.shape,y_train.shape
print(len(x_train), '训练数据')   #  训练数据 
print(len(x_val), '验证数据 ')    #  验证数据 
print(len(x_test), '测试数据')    #  测试数据
y_val

16170 训练数据
1774 验证数据 
1993 测试数据


array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -7.54126430e+03,  4.86492078e+06],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -7.63214360e+03,  4.86498222e+06],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -7.56419630e+03,  4.86488719e+06],
       ...,
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -7.53732190e+03,  4.86489578e+06],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -7.53640920e+03,  4.86493462e+06],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -7.51310449e+03,  4.86494387e+06]])

### building sae

In [5]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
model = Sequential()
sae_hidden_layers= [256,128,64,128,256]
model.add(Dense(sae_hidden_layers[0],input_dim=520,activation=tf.keras.layers.LeakyReLU(alpha=0.3),use_bias=False))
for units in sae_hidden_layers[1:]:
    model.add(Dense(units, activation=tf.keras.layers.LeakyReLU(alpha=0.3), use_bias=False))
model.add(Dense(520, activation=tf.keras.layers.LeakyReLU(alpha=0.3), use_bias=False))
model.compile(optimizer='adam', loss='mse')
model.fit(x_train,x_train,batch_size=64,epochs=20,verbose=1)

Epoch 1/20
253/253 [==============================] - 1s 2ms/step - loss: 0.2703
Epoch 2/20
253/253 [==============================] - 1s 2ms/step - loss: 0.1138
Epoch 3/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0865
Epoch 4/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0719
Epoch 5/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0626
Epoch 6/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0558
Epoch 7/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0512
Epoch 8/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0469
Epoch 9/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0437
Epoch 10/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0410
Epoch 11/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0387
Epoch 12/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0367
Epoch 13/20
253/253 [====

In [6]:
# path = './sae_model.hdf5'
# model.save(path)
num_to_remove = (len(sae_hidden_layers) + 1) // 2
for i in range(num_to_remove):
    model.pop()

### build and train a complete model with the trained SAE encoder and a classifier

In [7]:
classifier_hidden_layers = [128,128]
y_train_tem = y_train[:,:118]
y_val_tem = y_val[:,:118]
model.add(Dropout(0))
for units in classifier_hidden_layers:
    model.add(Dense(units, activation='relu', use_bias=False))
    model.add(Dropout(0))
model.add(Dense(OUTPUT_DIM, activation='sigmoid', use_bias=False))  # 'sigmoid' for multi-label classification
model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_tem, validation_data=(x_val, y_val_tem), batch_size=64, epochs=20, verbose=1)

Epoch 1/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0857 - accuracy: 0.8742 - val_loss: 0.0458 - val_accuracy: 0.8974
Epoch 2/20
253/253 [==============================] - 0s 2ms/step - loss: 0.0403 - accuracy: 0.7895 - val_loss: 0.0364 - val_accuracy: 0.7306
Epoch 3/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0334 - accuracy: 0.7083 - val_loss: 0.0318 - val_accuracy: 0.6759
Epoch 4/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0293 - accuracy: 0.6599 - val_loss: 0.0289 - val_accuracy: 0.6888
Epoch 5/20
253/253 [==============================] - 1s 2ms/step - loss: 0.0263 - accuracy: 0.6393 - val_loss: 0.0265 - val_accuracy: 0.6663
Epoch 6/20
253/253 [==============================] - 0s 2ms/step - loss: 0.0240 - accuracy: 0.6236 - val_loss: 0.0251 - val_accuracy: 0.6793
Epoch 7/20
253/253 [==============================] - 0s 2ms/step - loss: 0.0223 - accuracy: 0.6153 - val_loss: 0.0243 - val_accuracy: 0.5986
Epoch 

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               133120    
                                                                 
 dense_1 (Dense)             (None, 128)               32768     
                                                                 
 dense_2 (Dense)             (None, 64)                8192      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 128)               8192      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               1

In [8]:
preds = model.predict(x_test, batch_size=32)
b_results = np.equal(np.argmax(y_test[:,:3], axis=1), np.argmax(preds[:,:3], axis=1)).astype(int)
f_results = np.equal(np.argmax(y_test[:,3:8], axis=1), np.argmax(preds[:,3:8], axis=1)).astype(int)
r_results = np.equal(np.argmax(y_test[:,8:118], axis=1), np.argmax(preds[:,8:118], axis=1)).astype(int)
b_acc = b_results.mean()
f_acc = f_results.mean()
r_acc = r_results.mean()
acc = (b_results*f_results*r_results).mean()
b_acc,f_acc,r_acc,acc

63/63 [==============================] - 0s 903us/step


(0.9954841946813848,
 0.9909683893627697,
 0.6457601605619668,
 0.6422478675363773)

In [9]:
x_test_utm = np.asarray(y_test[:,-2])
y_test_utm = np.asarray(y_test[:,-1])
blds = preds[:,:3]
flrs = preds[:,3:8]
rfps = preds[:,8:118]
m = np.logical_and(b_results, f_results)  # mask index array for correct location of building and floor
mask = np.logical_and(m,r_results)
# x_test_utm = x_test_utm[mask]
# y_test_utm = y_test_utm[mask]
# blds = blds[mask]
# flrs = flrs[mask]
# print(len(r_results)) #2048
# print(np.sum(r_results == 0)) #704
# blds = (preds[mask])[:, :3]
# flrs = (preds[mask])[:, 3:8]
# rfps = (preds[mask])[:, 8:118]
blds,flrs,blds.shape,flrs.shape,rfps,rfps.shape,x_test_utm,y_test_utm

(array([[6.0824177e-06, 9.9240905e-01, 7.6874079e-07],
        [2.1692865e-04, 3.6515966e-01, 1.5080765e-04],
        [2.0748976e-15, 1.0000000e+00, 9.0971058e-05],
        ...,
        [4.1388871e-12, 4.2755353e-19, 1.0000000e+00],
        [5.7739535e-11, 2.6008790e-21, 1.0000000e+00],
        [6.8321668e-07, 9.9999386e-01, 3.8508688e-08]], dtype=float32),
 array([[1.2200667e-12, 2.8531115e-08, 9.9999118e-01, 4.8195259e-05,
         3.7412080e-11],
        [4.3870876e-09, 3.9372505e-09, 1.0000000e+00, 2.0650984e-11,
         4.9720210e-04],
        [8.2450269e-10, 1.9370773e-08, 9.9988216e-01, 1.6161532e-12,
         1.4673433e-11],
        ...,
        [1.3590647e-08, 1.0000000e+00, 4.1364659e-15, 2.3893831e-10,
         4.8825788e-09],
        [1.2691719e-06, 1.0000000e+00, 7.3330654e-11, 1.7981478e-16,
         1.7692686e-11],
        [6.3991803e-09, 2.2158546e-13, 7.0423549e-11, 1.0000000e+00,
         7.1523214e-09]], dtype=float32),
 (1993, 3),
 (1993, 5),
 array([[2.7405160e-06

In [10]:
n_success = len(blds)  # number of correct building and floor location
blds = np.greater_equal(blds, np.tile(np.amax(blds, axis=1).reshape(n_success, 1), (1, 3))).astype(
        int)  # set maximum column to 1 and others to 0 (row-wise)
flrs = np.greater_equal(flrs, np.tile(np.amax(flrs, axis=1).reshape(n_success, 1), (1, 5))).astype(int)
n_success,blds,flrs,blds.shape,flrs.shape,blds[1]

(1993,
 array([[0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0]]),
 array([[0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        ...,
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0]]),
 (1993, 3),
 (1993, 5),
 array([0, 1, 0]))

In [34]:
import math
n_loc_failure = 0
sum_pos_err = 0.0
sum_pos_err_weighted = 0.0
idxs = np.argpartition(rfps, -1)[:, -1:]  # (unsorted) indexes of up to N nearest neighbors
idxs,idxs[0],idxs.shape

(array([[11],
        [67],
        [49],
        ...,
        [86],
        [86],
        [18]], dtype=int64),
 array([11], dtype=int64),
 (1993, 1))

In [36]:
for i in range(n_success):
    xs = []
    ys = []
    ws = []
    for j in idxs[i]:
        rfp = np.zeros(110)
        rfp[j] = 1
        rows = np.where(
            (train_labels[:,:118] == np.concatenate((blds[i], flrs[i], rfp))).all(axis=1))  # tuple of row indexes
        if rows[0].size > 0:
            xs.append(train_df.iloc[rows[0][0], 520])  # LONGITUDE
            ys.append(train_df.iloc[rows[0][0], 521])  # LATITUDE
            ws.append(rfps[i][j])
        else:
            n_loc_failure += 1
        
    if len(xs) > 0:
        sum_pos_err += math.sqrt((np.mean(xs) - x_test_utm[i]) ** 2 + (np.mean(ys) - y_test_utm[i]) ** 2)
        sum_pos_err_weighted += math.sqrt(
                (np.average(xs, weights=ws) - x_test_utm[i]) ** 2 + (np.average(ys, weights=ws) - y_test_utm[i]) ** 2)
        xs.clear()
mean_pos_err = sum_pos_err / (n_success - n_loc_failure)
mean_pos_err_weighted = sum_pos_err_weighted / (n_success - n_loc_failure)
loc_failure = n_loc_failure / n_success  # rate of location estimation failure given that building and floor are correctly located
mean_pos_err,mean_pos_err_weighted,loc_failure,n_loc_failure

(3.899931278530471, 3.8999312785443765, 0.004515805318615153, 9)

In [13]:
rfp = np.zeros(110)
rfp[25] = 1
rows = np.where(
            (train_labels[:,:118] == np.concatenate((blds[0], flrs[0], rfp))).all(axis=1))
rfp,rows,rows[0].size,rows[0][0]

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 (array([  21,  133,  242,  352,  460,  567,  678,  789,  895, 1008],
        dtype=int64),),
 10,
 21)